# Function to Classify a Batch of Image Files Using a Trained Model #
First, define the function

In [55]:
##Classify image in image_path. Requires tensorflow to be installed on the system to work

#Function that labels an image using an existing tensorflow model:
##Inputs:
###image_path_list - list of full filepaths from the current working directory to an image to label. File type should be '.jpg' or '.png'
###model_graph_path - filepath to a tensorflow model graph to use for classification. File type should be '.pb'
##Outputs:
###Array with one column for each class and one row for each image with probability predictions for each class
def classify_image_batch(image_path_list, model_graph_path):
    
    #Import required packages
    import tensorflow as tf, sys
    import numpy as np
    #from PIL import Image
        
    ## Loads label file, strips off carriage return
    #label_lines = [line.rstrip() for line 
    #                   in tf.gfile.GFile(model_labels_path)]
    
    #Read image
    #imageread = Image.open(image_path)
    
    #Convert image to numpy array
    #image_array = np.array(imageread)[:,:,0:3]
    
    # Unpersists graph from file
    with tf.gfile.FastGFile(model_graph_path, 'rb') as f:
        graph_def = tf.GraphDef()
        graph_def.ParseFromString(f.read())
        _ = tf.import_graph_def(graph_def, name='')
    
    #Start tensorflow session
    #sess = tf.Session()
    with tf.Session() as sess:     
        # Define softmax tensor name
        softmax_tensor = sess.graph.get_tensor_by_name('final_result:0')
        
        #Read first image in to define array shape
        imagefile = tf.read_file(image_path_list[0])
        image_tensor = tf.image.decode_image(imagefile)
        image_array = image_tensor.eval()
        
        predictions = sess.run(softmax_tensor, \
                {'DecodeJpeg:0': image_array})
        
        output_array = np.zeros((len(image_path_list), np.shape(predictions)[1]))
        
        for i in range(0,len(image_path_list)):
        
            imagefile = tf.read_file(image_path_list[i])
            image_tensor = tf.image.decode_image(imagefile)
            image_array = image_tensor.eval()
        
            output_array[i,:] = sess.run(softmax_tensor, \
                    {'DecodeJpeg:0': image_array})[0]
    
    #Close tensorflow session to release resources
    #sess.close()
    
    return output_array

# Example Usage #

First, get a list of files in a particular subdirectory. Make sure it only contains valid .jpeg or .png files as other ones will cause the program to crash. Coerce them into a list of full filepaths from the current working directory.

In [121]:
filelist = os.listdir(path = "footballgood/football", )

In [122]:
filelist2 = filelist.copy()

for i in range(0,len(filelist)):
    filelist2[i] = 'footballgood/football/' + filelist[i]

In [123]:
filelist2

['footballgood/football/2017-04-0614-29-06football11.png',
 'footballgood/football/2017-04-0614-29-06football16.png',
 'footballgood/football/2017-04-0614-29-13football117.png',
 'footballgood/football/2017-04-0614-29-20football16.png',
 'footballgood/football/2017-04-0614-29-20football18.png',
 'footballgood/football/2017-04-0614-29-22football111.png',
 'footballgood/football/2017-04-0614-30-49football13.png',
 'footballgood/football/2017-04-0614-30-54football115.png',
 'footballgood/football/2017-04-0614-30-54football19.png',
 'footballgood/football/2017-04-0614-39-59football117.png',
 'footballgood/football/2017-04-0614-40-06football16.png',
 'footballgood/football/2017-04-0614-43-25football19.png',
 'footballgood/football/2017-04-0614-43-27football113.png',
 'footballgood/football/2017-04-0614-43-27football114.png',
 'footballgood/football/2017-04-0614-43-30football14.png',
 'footballgood/football/2017-04-0614-43-35football115.png',
 'footballgood/football/2017-04-0614-43-42footbal

Now run the batch classifier and examine the results. Note that labels are not currently included as part of the output.

In [124]:
batchout = classify_image_batch(image_path_list= filelist2, model_graph_path= 'output_graph_footballbig.pb')

In [125]:
batchout

array([[  3.18387210e-05,   9.99968171e-01],
       [  5.88207832e-03,   9.94117975e-01],
       [  1.32334651e-02,   9.86766577e-01],
       [  4.56886506e-03,   9.95431185e-01],
       [  5.15955221e-03,   9.94840443e-01],
       [  3.77567322e-03,   9.96224284e-01],
       [  1.05579186e-03,   9.98944223e-01],
       [  1.61866006e-03,   9.98381376e-01],
       [  3.33929923e-03,   9.96660709e-01],
       [  5.53814415e-03,   9.94461834e-01],
       [  1.98924681e-03,   9.98010814e-01],
       [  1.16544333e-03,   9.98834550e-01],
       [  6.41192403e-03,   9.93588090e-01],
       [  3.08394269e-03,   9.96916056e-01],
       [  1.42714917e-03,   9.98572826e-01],
       [  4.41002334e-03,   9.95589972e-01],
       [  7.98780564e-03,   9.92012203e-01],
       [  6.55886251e-03,   9.93441105e-01],
       [  9.54741299e-01,   4.52586971e-02],
       [  9.58451536e-03,   9.90415454e-01],
       [  6.69316156e-03,   9.93306816e-01],
       [  8.20031017e-03,   9.91799712e-01],
       [  

In [119]:
for i in range(0,len(batchout)):
    
    if batchout[i,0] < batchout[i,1]:
        print(i)
        print(batchout[i,:])
        print(filelist2[i])

18
[ 0.3728978  0.6271022]
footballgood/good/2017-04-0614-51-20football19.png
24
[ 0.42925912  0.57074082]
footballgood/good/2017-04-0614-53-53football110.png
36
[ 0.28074372  0.71925634]
footballgood/good/2017-04-0615-02-31football112.png


In [120]:
goodresults = batchout.copy()